In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset,num2date
from sklearn.cluster import KMeans

In [2]:
def classifiability(data,ncluster,nsim):
    """
    An implementation of the Michelangeli et al. 1995 JAS routine
    (equation 3.2)
    
    In PC space, for each value of k, for ntrials of kmeans clustering with different random seeds:
    1. Correlates each cluster centroid with each other cluster centroid.
    2. For each k in partition P, finds its best fit (max correlation) in Q.
    3. Finds the lowest of these (the cluster in P with the worst fit in Q).
    4. Computes the average of these over ntrials, yielding the classifiability.
    
    Michelangeli used 50 iterations.
    
    The notation follows M95 for P, Q, Aij, Aprime, and c_pq
    """
        
    centroids = np.full((nsim,ncluster,data.shape[1]),np.nan)
    rand_seeds = np.random.choice(np.arange(1000),nsim,replace=False) ## these need to be unique seeds

    for n in range(nsim):
        ## run the kmeans algorithm once from specifed seed and store centroids
        kmeans_trial = KMeans(n_clusters=ncluster, n_init=1, max_iter=500,random_state=rand_seeds[n]).fit(data)
        centroids[n] = kmeans_trial.cluster_centers_

    c_pq = np.full((nsim,nsim),np.nan)

    for i in range(nsim):
        for j in range(nsim):
            if i!=j:    

                P = centroids[i]; Q = centroids[j]

                Aij = np.full((ncluster,ncluster),np.nan)
                for r1 in range(ncluster):
                    for r2 in range(ncluster):
                        Aij[r1,r2]=np.corrcoef(P[r1],Q[r2])[0,1]

                Aprime=np.max(Aij,axis=0)
                c_pq[i,j] = np.min(Aprime)

    classif=np.nanmean(c_pq)
    print(classif,flush=True)
    
    return classif